In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from timeit import default_timer as timer
import illustris_python as il
import physics

#intitial setup
basePath = "../data/tng-100-3/output"
fields = {"stars": ["Coordinates", "Potential", "Masses", "Velocities"],
        "gas": ["Coordinates", "Potential", "Masses", "Velocities"],
        "dm": ["Coordinates", "Potential", "Velocities"]
        }
indices = np.genfromtxt("../data/tng-100-3/cutdata/central_ids.txt")

#indices[0] = 1893

#[544, 953, 1285, 1662]
#[1893, 2164, 3043, 3229, 3403, 3605, 3760] 
#np.genfromtxt("../data/tng-100-3/cutdata/central_ids.txt")
DM_PARTICLE_MASS = 0.03235675 #found in header of snapshot
N = len(indices)
stars = [0]*N
gas = [0]*N
dm = [0]*N
particle_lists = [gas, dm, stars]
group_cat = pd.DataFrame({"id": indices})
#Start timer
start = timer()


In [2]:
#Load all particles
for i in range(N):
    stars[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'stars', fields["stars"]))
    gas[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'gas', fields["gas"]))
    dm[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'dm', fields["dm"]))
    dm[i]["Masses"] = [DM_PARTICLE_MASS]*len(dm[i]["Potential"]) #Add dm masses

subhaloFields = ["SubhaloMass", 'SubhaloMassType', "SubhaloMassInHalfRadType", "SubhaloHalfmassRadType", "SubhaloPos", "SubhaloVel", "SubhaloSpin"]
subhalos = il.groupcat.loadSubhalos(basePath, 99, fields=subhaloFields)
df_subhalos = il.pandasformat.dict_to_pandas(subhalos)

In [3]:
def max_ang_momentum(particle, N, catalogue):
    #max_h_dir = np.zeros([N, 3])
    h_avg_dir = np.zeros([N, 3])
    for i in range(N):
        r_max = 5*catalogue["SubhaloHalfmassRadStellar"][i]
        temp = particle[i][particle[i]["r"] < r_max].copy(deep=True)
        h_len = np.zeros(len(temp["r"]))
        v = np.array([np.array(temp["Vx"]), np.array(temp["Vy"]), np.array(temp["Vz"])])
        r = np.array([np.array(temp["x"]), np.array(temp["y"]), np.array(temp["z"])])
        h = np.cross(np.transpose(r), np.transpose(v))
        for j in range (len(h)):
            h_len[j] = np.linalg.norm(h[j])
        #h_max, max_index = h_len.max(), h_len.argmax()
        #print(max_index, len(h))
        #max_h_dir[i] = [h[max_index][0], h[max_index][1], h[max_index][2]]/h_max
        #print(v[0][L_max_index], v[1][L_max_index], v[2][L_max_index])
        h_avg = [h[:,0].mean(), h[:,1].mean(), h[:,2].mean()]
        h_avg_dir[i] = h_avg/np.linalg.norm(h_avg)
 
    catalogue["RotationAxisX"] = [item[0] for item in h_avg_dir]
    catalogue["RotationAxisY"] = [item[1] for item in h_avg_dir]
    catalogue["RotationAxisZ"] = [item[2] for item in h_avg_dir]
    return catalogue

In [4]:
print("Calculating relative positions and radius")
particle_lists, group_cat = physics.properties.relative_pos_radius(particle_lists, N, group_cat)
print("Calculating particle masses")
particle_lists, group_cat = physics.properties.total_mass(particle_lists, N, group_cat)
print("Calculating subhalo velocity")
particle_lists, group_cat = physics.properties.subhalo_velocity(particle_lists, N, group_cat)
print("Calculating relative velocities")
for particle in particle_lists:
    particle = physics.properties.relative_velocities(particle, N, group_cat)
print("Calculating half mass radius")
stars, group_cat = physics.properties.half_mass_radius(stars, N, group_cat, "Stellar")
print("Calculating maximum angular momentum")
group_cat = max_ang_momentum(stars, N, group_cat)
rot_vector = np.transpose(np.array([group_cat["RotationAxisX"], group_cat["RotationAxisY"], group_cat["RotationAxisZ"]]))
print("Rotating subhalos")
particle_lists = physics.geometry.rotate_coordinates(particle_lists, N, group_cat, rot_vector)
#End timer
end = timer()

Calculating relative positions and radius
Calculating particle masses
Calculating subhalo velocity
Calculating relative velocities
Calculating half mass radius
Calculating maximum angular momentum
Rotating subhalos


In [5]:
print("Masses check")
for i in range(3):
    print("Group catalogue: ", df_subhalos["SubhaloMassStellar"][indices[i]], " Snapshot: ", group_cat["SubhaloMassStellar"][i])

Masses check
Group catalogue:  80.1757  Snapshot:  80.17588806152344
Group catalogue:  90.369896  Snapshot:  90.3699951171875
Group catalogue:  43.96099  Snapshot:  43.97459030151367


In [6]:
print("Velocities check")
for i in range(3):
    print(group_cat["SubhaloVelocityX"][i], group_cat["SubhaloVelocityY"][i], group_cat["SubhaloVelocityZ"][i])
    print(df_subhalos["SubhaloVel"][indices[i]])

Velocities check
-175.11496411068748 81.26290417921925 -8.336148174927477
[-175.13785    81.367256   -8.339443]
279.265925652316 -99.84715851410176 82.07070970468858
[279.23257  -99.84879   82.072495]
174.67770423321207 57.30106101855136 170.3806483246425
[174.64732  57.31744 170.41402]


In [7]:
print("Halfmass radius check")
for i in range(3): 
    print("Group catalogue: ", df_subhalos["SubhaloHalfmassRadStellar"][indices[i]], " Snapshot: ", group_cat["SubhaloHalfmassRadStellar"][i])

Halfmass radius check
Group catalogue:  64.70497  Snapshot:  57.03927079687098
Group catalogue:  79.047134  Snapshot:  81.96062358633587
Group catalogue:  33.925236  Snapshot:  43.439945028308486


In [8]:
print("Spin direction  check")
for i in range(3):
    print("Subhalo: ", i)
    s_vec = np.array(df_subhalos["SubhaloSpin"][indices[i]])
    s = np.linalg.norm(s_vec)
    s_dir = s_vec/s
    print("Snapshot : ", rot_vector[i])
    print("catalogue: ", s_dir)

Spin direction  check
Subhalo:  0
Snapshot :  [-0.014549    0.4492872   0.89326891]
catalogue:  [ 0.21661627 -0.66537654 -0.7143889 ]
Subhalo:  1
Snapshot :  [ 0.5662872  -0.74470173  0.35318288]
catalogue:  [ 0.18161198 -0.07396439 -0.9805847 ]
Subhalo:  2
Snapshot :  [-0.21417035 -0.66251506  0.71777772]
catalogue:  [-0.28974828 -0.75167984  0.5924722 ]


In [9]:
print(gas[0]["Vx"][0])
print(stars[0]["Vx"][0])

-273.7644
-58.701385


In [10]:
#Save all fields for particles in one subhalo for later inspection
g = 0
stars[g].to_pickle("../data/tng-100-3/subhalos/subhalo" + str(indices[g]) +"_stars.pkl")

In [11]:
#Save group catalogue
group_cat.to_pickle("../data/tng-100-3/catalogues/test_cat_01.pkl")

In [12]:

print("Time to process " + str(N) + " Subhalos: ")
print(int(end - start), "Seconds")

Time to process 104 Subhalos: 
193 Seconds
